In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Read Data

In [ ]:
df = pd.read_csv('../data/raw/train_period_1.csv')
df.head()

In [ ]:
df.columns

In [ ]:
df.info()

## Processing

In [ ]:
df_processed = df.copy()

### Handle Mising Values

In [ ]:
print(df_processed.isnull().sum())

In [ ]:
# Drop rows with missing values
df_processed = df_processed.dropna()

In [ ]:
print(df_processed.isnull().sum())

### Handle Duplicated Value

In [ ]:
# Count how many rows are duplicates based on CustomerID
duplicate_count = df_processed.duplicated(subset=['CustomerID']).sum()
print(f"Number of duplicate CustomerIDs: {duplicate_count}")

# View the actual duplicate rows
duplicates = df_processed[df_processed.duplicated(subset=['CustomerID'], keep=False)]
print(duplicates.sort_values(by='CustomerID').head())

In [ ]:
# Keep the first occurrence and remove subsequent ones
df_processed = df_processed.drop_duplicates(subset=['CustomerID'], keep='first')

### Convert CustomerID to string

In [ ]:
df_processed['CustomerID'] = df_processed['CustomerID'].astype("int64")

In [ ]:
df_processed.info()

### Convert float columns that should be integers

In [ ]:
int_columns = ['Age', 'Tenure', 'Support Calls', 'Last Interaction']
for col in int_columns:
    df_processed[col] = df_processed[col].astype(int)

### Handle categorical variables

In [ ]:
categorical_features  = ['Gender', 'Subscription Type', 'Contract Length']
for col in categorical_features :
    print(f"{col}: {df_processed[col].unique()}")

In [ ]:
# We use options A because the models we train is Random Forest

In [ ]:
# Option A: One-hot encoding (for tree-based models)
df_encoded = pd.get_dummies(df_processed, columns=categorical_features , drop_first=True)

In [ ]:
# OR Option B: Label encoding (for linear models)
# label_encoders = {}
# for col in categorical_cols:
#     le = LabelEncoder()
#     df[col] = le.fit_transform(df[col])
#     label_encoders[col] = le

### Check outliers

In [ ]:
numerical_features  = ['Age', 'Tenure', 'Usage Frequency', 'Support Calls', 
                  'Payment Delay', 'Total Spend']

In [ ]:
for col in numerical_features :
    Q1 = df_processed[col].quantile(0.25)
    Q3 = df_processed[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df_processed[(df_processed[col] < lower_bound) | (df_processed[col] > upper_bound)]
    print(f"{col}: {len(outliers)} outliers ({len(outliers)/len(df_processed)*100:.2f}%)")

In [ ]:
# Because the dataset don't have outliers, we will not remove any rows
# If needed, we could use:
# Handle outliers (if any)
# Option 1: Cap outliers
# for col in numerical_cols:
#     Q1 = df[col].quantile(0.25)
#     Q3 = df[col].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     df[col] = np.clip(df[col], lower_bound, upper_bound)

# Option 2: Remove outliers
# for col in numerical_cols:
#     Q1 = df[col].quantile(0.25)
#     Q3 = df[col].quantile(0.75)
#     IQR = Q3 - Q1
#     df = df[(df[col] >= Q1 - 1.5*IQR) & (df[col] <= Q3 + 1.5*IQR)]

### Create Feature 

In [ ]:
df_processed['Tenure_Age_Ratio'] = df_processed['Tenure'] / (df_processed['Age'] + 1)
df_processed['Spend_per_Usage'] = df_processed['Total Spend'] / (df_processed['Usage Frequency'] + 1)
df_processed['Support_Calls_per_Tenure'] = df_processed['Support Calls'] / (df_processed['Tenure'] + 1)

In [ ]:
# Create customer segments based on spending
df_processed['Spending_Group'] = pd.qcut(df_processed['Total Spend'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])

In [ ]:
# Create tenure groups
df_processed['Tenure_Group'] = pd.cut(df_processed['Tenure'], 
                                      bins=[0, 12, 24, 36, 100], 
                                      labels=['<1yr', '1-2yr', '2-3yr', '3+yr'])

# Add categorical features to the list
categorical_features.extend(['Spending_Group', 'Tenure_Group'])
df_processed.head()

In [ ]:
import os

out_dir = '../data/processed'
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, 'df_processed.csv')

df_processed.to_csv(out_path, index=False)
print(f"Exported df_processed to {out_path}")

## EDA

In [ ]:
plt.plot()
df['Churn'].value_counts().plot(kind='bar', color=['skyblue', 'salmon'])
plt.title('Churn Distribution')
plt.xlabel('Churn')
plt.ylabel('Count')
plt.xticks([0, 1], ['Not Churned', 'Churned'], rotation=0)

In [ ]:
plt.plot()
sns.boxplot(x='Churn', y='Age', data=df)
plt.title('Age Distribution by Churn')

In [ ]:
plt.plot()
sns.boxplot(x='Churn', y='Tenure', data=df)
plt.title('Tenure Distribution by Churn')

In [ ]:
plt.plot()
numeric_df = df.select_dtypes(include=[np.number])
correlation_matrix = numeric_df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', center=0)
plt.title('Correlation Heatmap')

In [ ]:
plt.plot()
sns.boxplot(x='Churn', y='Payment Delay', data=df)
plt.title('Payment Delay by Churn')

In [ ]:
plt.plot(2, 3, 6)
sns.boxplot(x='Churn', y='Total Spend', data=df)
plt.title('Total Spend by Churn')

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
# Gender vs Churn
gender_churn = pd.crosstab(df['Gender'], df['Churn'])
gender_churn.plot(kind='bar', ax=axes[0, 0])
axes[0, 0].set_title('Churn by Gender')
axes[0, 0].set_xlabel('Gender')
axes[0, 0].set_ylabel('Count')

# Subscription Type vs Churn
sub_churn = pd.crosstab(df['Subscription Type'], df['Churn'])
sub_churn.plot(kind='bar', ax=axes[0, 1])
axes[0, 1].set_title('Churn by Subscription Type')
axes[0, 1].set_xlabel('Subscription Type')
axes[0, 1].set_ylabel('Count')

# Contract Length vs Churn
contract_churn = pd.crosstab(df['Contract Length'], df['Churn'])
contract_churn.plot(kind='bar', ax=axes[1, 0])
axes[1, 0].set_title('Churn by Contract Length')
axes[1, 0].set_xlabel('Contract Length')
axes[1, 0].set_ylabel('Count')

# Support Calls vs Churn
support_churn = pd.crosstab(pd.cut(df['Support Calls'], bins=5), df['Churn'])
support_churn.plot(kind='bar', ax=axes[1, 1])
axes[1, 1].set_title('Churn by Support Calls (Binned)')
axes[1, 1].set_xlabel('Support Calls')
axes[1, 1].set_ylabel('Count')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()